In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar -xf /content/drive/MyDrive/skin_cancer/final_data.tar -C /content/

In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ConvNeXtTiny
from tensorflow.keras.applications.convnext import preprocess_input
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.losses import CategoricalFocalCrossentropy

# PATHS
TRAIN_DIR = "/content/final_data/train"
VAL_DIR = "/content/final_data/valid/sorted"
TEST_DIR = "/content/final_data/test/sorted"
LOCAL_CHECKPOINT_PATH = "/content/drive/MyDrive/skin_cancer/models/convnext_tiny_checkpoints"
FINAL_MODEL_PATH = "/content/drive/MyDrive/skin_cancer/models/convnext_tiny_full_model.h5"
os.makedirs(LOCAL_CHECKPOINT_PATH, exist_ok=True)
os.makedirs(os.path.dirname(FINAL_MODEL_PATH), exist_ok=True)

# PARAMETERS
IMG_SIZE = (224, 224)   # ConvNeXt default input size
BATCH_SIZE = 32
EPOCHS = 30

# CENTER CROP FUNCTION
def center_crop_and_preprocess(img):
    """Crop to center square, resize, then preprocess for ConvNeXt."""
    h, w, _ = img.shape
    min_side = min(h, w)
    top = (h - min_side) // 2
    left = (w - min_side) // 2
    img = img[top:top + min_side, left:left + min_side]
    img = tf.image.resize(img, IMG_SIZE)
    img = preprocess_input(img)
    return img

# DATA AUGMENTATION
train_datagen = ImageDataGenerator(
    preprocessing_function=center_crop_and_preprocess,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    shear_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest",
)

val_datagen = ImageDataGenerator(preprocessing_function=center_crop_and_preprocess)
test_datagen = ImageDataGenerator(preprocessing_function=center_crop_and_preprocess)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=True
)
val_gen = val_datagen.flow_from_directory(
    VAL_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)
test_gen = test_datagen.flow_from_directory(
    TEST_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

# CLASS WEIGHTS → for Focal Loss alpha
labels = train_gen.classes

class_counts = np.bincount(labels)
print("Counts per class:", class_counts)

# inverse-sqrt weighting
alpha_array = 1.0 / np.sqrt(class_counts)
alpha_array = alpha_array / np.sum(alpha_array)
print("New alpha_array:", alpha_array)

# MODEL (ConvNeXt-Tiny)
base_model = ConvNeXtTiny(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze base model (warmup phase)
for layer in base_model.layers:
    layer.trainable = False

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dropout(0.4)(x)
x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
output = layers.Dense(train_gen.num_classes, activation="softmax")(x)

model = models.Model(inputs=base_model.input, outputs=output)

# LOSS FUNCTION
loss_fn = CategoricalFocalCrossentropy(
    gamma=1.8,
    alpha=alpha_array,
    label_smoothing=0.05,
    from_logits=False
)

# COMPILE (Warmup Phase)
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss=loss_fn,
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), AUC(name="auc")]
)

# CALLBACKS
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(LOCAL_CHECKPOINT_PATH, "ckpt-{epoch:02d}.keras"),
    save_weights_only=False,
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)
early_stop = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=4, min_lr=1e-6, verbose=1)

# PHASE 1: Warmup (Top Layers)
history_warmup = model.fit(
    train_gen,
    epochs=2,
    validation_data=val_gen,
    callbacks=[checkpoint_callback, early_stop, reduce_lr],
    verbose=1
)

# PHASE 2: Fine-tuning deeper layers
fine_tune_at = len(base_model.layers) // 2
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
for layer in base_model.layers[fine_tune_at:]:
    layer.trainable = True

model.compile(
    optimizer=optimizers.Adam(learning_rate=5e-5),
    loss=loss_fn,
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), AUC(name="auc")]
)

history_finetune = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    initial_epoch=2,
    callbacks=[checkpoint_callback, early_stop, reduce_lr],
    verbose=1
)

# SAVE FINAL MODEL
model.save(FINAL_MODEL_PATH, save_format="tf")
print(f"✅ Training complete! Full model saved at {FINAL_MODEL_PATH}")

Found 27205 images belonging to 7 classes.
Found 235 images belonging to 7 classes.
Found 1470 images belonging to 7 classes.
Counts per class: [3000 3500 4000 3000 4000 6705 3000]
New alpha_array: [0.15805851 0.14633375 0.13688269 0.15805851 0.13688269 0.10572535
 0.15805851]
111650432/111650432 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
851/851 ━━━━━━━━━━━━━━━━━━━━ 0s 640ms/step - accuracy: 0.2560 - auc: 0.6311 - loss: 0.3795 - precision: 0.2999 - recall: 0.1520
Epoch 1: val_loss improved from inf to 0.23630, saving model to /content/drive/MyDrive/skin_cancer/models/convnext_tiny_checkpoints/ckpt-01.keras
851/851 ━━━━━━━━━━━━━━━━━━━━ 583s 658ms/step - accuracy: 0.2560 - auc: 0.6312 - loss: 0.3794 - precision: 0.2999 - recall: 0.1521 - val_accuracy: 0.5745 - val_auc: 0.8617 - val_loss: 0.2363 - val_precision: 0.7874 - val_recall: 0.4255 - learning_rate: 1.0000e-04
Epoch 2/2
851/851 ━━━━━━━━━━━━━━━━━━━━ 0s 619ms/step - accuracy: 0.3927 - auc: 0.7716 - loss: 0.2911 - precision: 0.4958 - recall: 0.2595
Epoch 2: val_loss improved from 0.23630 to 0.22302, saving model to /content/drive/MyDrive/skin_cancer/models/convnext_tiny_checkpoints/ckpt-02.keras
851/851 ━━━━━━━━━━━━━━━━━━━━ 531s 623ms/step - accuracy: 0.3927 - auc: 0.7716 - loss: 0.2911 - precision: 0.4959 - recall: 0.2595 - val_accuracy: 0.6213 - val_auc: 0

In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ConvNeXtTiny
from tensorflow.keras.applications.convnext import preprocess_input
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.losses import CategoricalFocalCrossentropy

# PATHS
TRAIN_DIR = "/content/final_data/train"
VAL_DIR = "/content/final_data/valid/sorted"
TEST_DIR = "/content/final_data/test/sorted"
LOCAL_CHECKPOINT_PATH = "/content/drive/MyDrive/skin_cancer/models/convnext_tiny_checkpoints"
FINAL_MODEL_PATH = "/content/drive/MyDrive/skin_cancer/models/convnext_tiny_full_model.h5"
os.makedirs(LOCAL_CHECKPOINT_PATH, exist_ok=True)
os.makedirs(os.path.dirname(FINAL_MODEL_PATH), exist_ok=True)

# PARAMETERS
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 30

# CENTER CROP FUNCTION
def center_crop_and_preprocess(img):
    """Crop to center square, resize, then preprocess for ConvNeXt."""
    h, w, _ = img.shape
    min_side = min(h, w)
    top = (h - min_side) // 2
    left = (w - min_side) // 2
    img = img[top:top + min_side, left:left + min_side]
    img = tf.image.resize(img, IMG_SIZE)
    img = preprocess_input(img)
    return img
# DATA AUGMENTATION
train_datagen = ImageDataGenerator(
    preprocessing_function=center_crop_and_preprocess,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    shear_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest",
)

val_datagen = ImageDataGenerator(preprocessing_function=center_crop_and_preprocess)
test_datagen = ImageDataGenerator(preprocessing_function=center_crop_and_preprocess)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=True
)
val_gen = val_datagen.flow_from_directory(
    VAL_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)
test_gen = test_datagen.flow_from_directory(
    TEST_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

# CLASS WEIGHTS → for Focal Loss alpha
labels = train_gen.classes

class_counts = np.bincount(labels)
print("Counts per class:", class_counts)

# inverse-sqrt weighting
alpha_array = 1.0 / np.power(class_counts, 0.6)
alpha_array = alpha_array / np.sum(alpha_array)
print("New alpha_array:", alpha_array)

# boost specific classes manually
boost = {
    0: 1.4,
    1: 1.3,
    4: 1.5,   # MEL index (increase focus)
    5: 0.7,   # NV index (slightly reduce)
}

for idx, factor in boost.items():
    alpha_array[idx] *= factor

# re-normalize after boosting
alpha_array = alpha_array / np.sum(alpha_array)

print("Adjusted alpha_array:", alpha_array)

# LOSS FUNCTION
loss_fn = CategoricalFocalCrossentropy(
    gamma=1.8,
    alpha=alpha_array,
    label_smoothing=0.05,
    from_logits=False
)

# CALLBACKS
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(LOCAL_CHECKPOINT_PATH, "ckpt-{epoch:02d}.keras"),
    save_weights_only=False,
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)
early_stop = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=4, min_lr=1e-6, verbose=1)


model = tf.keras.models.load_model('/content/drive/MyDrive/skin_cancer/models/convnext_tiny_checkpoints/ckpt-18.keras', compile=False)
# Re-compile with smaller LR

model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-5),
    loss=loss_fn,
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), AUC(name="auc")]
)

history_finetune = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    initial_epoch=18,
    callbacks=[checkpoint_callback, early_stop, reduce_lr],
    verbose=1
)

# SAVE FINAL MODEL
model.save(FINAL_MODEL_PATH, save_format="tf")
print(f"✅ Training complete! Full model saved at {FINAL_MODEL_PATH}")

Found 27205 images belonging to 7 classes.
Found 235 images belonging to 7 classes.
Found 1470 images belonging to 7 classes.
Counts per class: [3000 3500 4000 3000 4000 6705 3000]
New alpha_array: [0.1609839  0.14676225 0.13546253 0.1609839  0.13546253 0.09936099
 0.1609839 ]
Adjusted alpha_array: [0.19660522 0.16643409 0.11816906 0.1404323  0.17725359 0.06067343
 0.1404323 ]


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 19/30
851/851 ━━━━━━━━━━━━━━━━━━━━ 0s 940ms/step - accuracy: 0.9275 - auc: 0.9943 - loss: 0.0954 - precision: 0.9526 - recall: 0.8756
Epoch 19: val_loss improved from inf to 0.12337, saving model to /content/drive/MyDrive/skin_cancer/models/convnext_tiny_checkpoints/ckpt-19.keras
851/851 ━━━━━━━━━━━━━━━━━━━━ 857s 962ms/step - accuracy: 0.9275 - auc: 0.9943 - loss: 0.0954 - precision: 0.9526 - recall: 0.8756 - val_accuracy: 0.7702 - val_auc: 0.9568 - val_loss: 0.1234 - val_precision: 0.8104 - val_recall: 0.7277 - learning_rate: 2.0000e-05
Epoch 20/30
851/851 ━━━━━━━━━━━━━━━━━━━━ 0s 889ms/step - accuracy: 0.9288 - auc: 0.9946 - loss: 0.0922 - precision: 0.9520 - recall: 0.8776
Epoch 20: val_loss did not improve from 0.12337
851/851 ━━━━━━━━━━━━━━━━━━━━ 760s 892ms/step - accuracy: 0.9288 - auc: 0.9946 - loss: 0.0922 - precision: 0.9520 - recall: 0.8776 - val_accuracy: 0.7064 - val_auc: 0.9355 - val_loss: 0.1353 - val_precision: 0.7729 - val_recall: 0.6809 - learning_rate: 2.0000e-05

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.convnext import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# Paths
CHECKPOINT_PATH = "/content/drive/MyDrive/skin_cancer/models/convnext_tiny_checkpoints/ckpt-21.keras"
TEST_DIR = "/content/final_data/test/sorted"
IMG_SIZE = (224, 224)  # Use same as training
BATCH_SIZE = 32

# Preprocessing and Generators
def center_crop_and_preprocess(img):
    """Crop to center square, resize, then preprocess for ConvNeXt."""
    h, w, _ = img.shape
    min_side = min(h, w)
    top = (h - min_side) // 2
    left = (w - min_side) // 2
    img = img[top:top + min_side, left:left + min_side]
    img = tf.image.resize(img, IMG_SIZE)
    img = preprocess_input(img)
    return img

test_datagen = ImageDataGenerator(preprocessing_function=center_crop_and_preprocess)

test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Load Model & Evaluate
model = tf.keras.models.load_model(CHECKPOINT_PATH, compile=False)

steps = int(np.ceil(test_gen.samples / test_gen.batch_size))
preds = model.predict(test_gen, steps=steps, verbose=1)

# Evaluation Metrics
y_true = test_gen.classes
y_pred = np.argmax(preds, axis=1)
target_names = list(test_gen.class_indices.keys())

print("\n===== Classification Report =====")
print(classification_report(y_true, y_pred, target_names=target_names, digits=4))

macro_f1 = f1_score(y_true, y_pred, average='macro')
print("Macro F1:", round(macro_f1, 4))

print("\n===== Confusion Matrix =====")
print(confusion_matrix(y_true, y_pred))

Found 1470 images belonging to 7 classes.
46/46 ━━━━━━━━━━━━━━━━━━━━ 26s 465ms/step

===== Classification Report =====
              precision    recall  f1-score   support

       AKIEC     0.6364    0.4667    0.5385        30
         BCC     0.7821    0.6559    0.7135        93
         BKL     0.6000    0.8295    0.6963       217
          DF     0.6786    0.7600    0.7170        25
         MEL     0.5312    0.4971    0.5136       171
          NV     0.9214    0.8636    0.8915       909
        VASC     0.7333    0.8800    0.8000        25

    accuracy                         0.7932      1470
   macro avg     0.6976    0.7075    0.6958      1470
weighted avg     0.8066    0.7932    0.7958      1470

Macro F1: 0.6958

===== Confusion Matrix =====
[[ 14   0  14   0   2   0   0]
 [  2  61  18   2   3   5   2]
 [  2   5 180   3  13  11   3]
 [  0   1   2  19   0   3   0]
 [  2   3  32   0  85  47   2]
 [  2   7  54   3  57 785   1]
 [  0   1   0   1   0   1  22]]
